# Use SQL in Python

## 1. Use Python Library

### 1) Import library and connection

In [1]:
import ibm_db

ModuleNotFoundError: No module named 'ibm_db'

In [ ]:

dsn_hostname = "xxxxxx" 
dsn_uid = "xxxxxx"       
dsn_pwd = "xxxxxx"     

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"


In [ ]:
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

print(dsn)

### 2) Connection to DB

In [ ]:
try:
    conn = ibm_db.connect(dsn,"","")
    print('Connected to DB: ', dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print("Unable to connect: ", ibm_db.conn_errormsg())


In [ ]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

In [ ]:
#Retrieve Metadata for the Database Client / Driver
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

### 3) Start queries

In [2]:
#drop
dropQuery = "DROP table INSTRUCTOR"

dropStmt = ibm_db.exec_immediate(conn, dropQuery)

NameError: name 'ibm_db' is not defined

In [3]:
#create

createQuery = "CREATE table INSTRUCTOR(id INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(15), CCODE CHAR(2))"

createStmt = ibm_db.exec_immediate(conn, createQuery)

NameError: name 'ibm_db' is not defined

In [4]:
# insert data

insertQuery = "INSERT INTO INSTRUCTOR(id, FNAME, LNAME, CITY, CCODE) VALUES (1, 'Ahuja', 'Rav','Toronto', 'CA')"

insertStmt = ibm_db.exec_immediate(conn, insertQuery)

insertQuery2 = "INSERT into INSTRUCTOR VALUES(2, 'Chong', 'Raul','Toronto', 'CA'),(3, 'Vasuadevan','Hima','Chicago', 'US')"

insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)

NameError: name 'ibm_db' is not defined

In [5]:
# retrieves all rows

selectQuery = "SELECT * from INSTRUCTOR"

selectStmt = ibm_db.exec_immediate(conn, selectQuery)

### 4) Fetch data

In [ ]:
# fetch the Dictionary only the first row only

ibm_db.fetch_both(selectStmt)

In [ ]:
# fetch the rest

whilte ibm_db.fetch_row(selectStmt)!= False:
    print('ID is: ', ibm_db.result(selectStmt, 0), "first name is: ", ibm_db.result(selectStmt, 'FNAME'))

### 5) Change data

In [ ]:
changeQuery = "UPDATE INSTRUCTOR SET CITY = 'MOOSETOWN' where FNAME = 'Rav'"

changeStmt = ibm_db.exec_immediate(conn, changeQuery)

### 6) Retrieve data to Pandas

In [ ]:
import pandas as pd
import ibm_db_dbi

# establish connection for pd
pconn = ibm_db_dbi.Connection(conn)


In [ ]:
# read in sql

df = pd.read_sql(selectQuery,pconn)

df.head(2)

df.FNAME[1]

### 7) Close connection

In [ ]:
ibm_db.close(conn)

## 2. Use SQL Extension in Python

### 1) Load SQL extension and connection

In [6]:
# load sql extension
%load_ext sql

# connection
%sql ibm_db_sa://gfs18179:8%40hf79m5h32t23x9@dashdb-txn-sbox-yp-dal09-12.services.dal.bluemix.net:50000/BLUDB

ModuleNotFoundError: No module named 'sql'

### 2) From pandas to SQL (persist)

In [ ]:
chicago_soceco = pd.read_csv("https://data.cityofchicago.org/resource/jcxq-k9xf.csv") 

# read in with "PERSIST" from pandas
%sql PERSIST chicage_soceco


### 3) Queries

In [ ]:
# first five rows
%sql SELECT * from chicago_soceco limit 5;

# number of rows
%sql SELECT COUNT(*) from chicago_soceco

%sql SELECT COUNT(*) from chicago_sociec where hardship_index >50.0;

%sql SELECT MAX(hardship_index) from chicago_sociec;

# two ways to select the area with highest hardship_index
%sql SELECT ca, community_area_name from chicago_soceco where hardship_index = (SELECT MAX(hardship_index) FROM chicago_soceco);
%sql SELECT community_area_name from chicago_soceco ORDER BY hardship_index DESC NULLS LAST FETCH FIRST ROW ONLY;

%sql select community_area_name from chicago_soceco where per_capita_income_ > 60000;


### 4) Plot SQL to DF (no close)

In [ ]:
# Plot two columns from a SQL DB
import seaborn as sns
%matplotlib inline

per_capita_income_vs_hardship_index = %sql SELECT per_capita_income_, hardship_index from chicago_sociec

plot = sns.jointplot(x = 'per_capita_income_', y = 'hardship_index',
                     data = per_capita_income_vs_hardship_index.DataFrame()) # notice the conversion to DF

## 3. More Examples

### 1) Check table properties

In [ ]:
%load_ext sql
%sql ibm_db_sa://xxxxxx:8%40xxxxxx@dashdb-txn-sbox-yp-dal09-12.services.dal.bluemix.net:50000/BLUDB
            
# what tables do you have            
%sql select TABSCHEMA, TABNAME,create_time from syscat.tables where tabschema = 'GFS18179'

# how many columns in a specific table
%sql select count(*) from syscat.columns where tabname = 'CHICAGO_PUBLIC_SCHOOLS'

# get the col_properties in two ways
%sql select distinct(name), coltype, length from sysibm.syscolumns where tbname = 'CHICAGO_PUBLIC_SCHOOLS'
%sql select COLNAME, TYPENAME, LENGTH from syscat.columns where tabname = 'CHICAGO_PUBLIC_SCHOOLS'

### 2) One table queries

In [7]:
# how many elementary schools
%sql select COUNT(*) from CHICAGO_PUBLIC_SCHOOLS WHERE "Elementary, Middle, or High School" = 'ES

# what is the highest Safety Score? which school?
%sql select MAX(SAFETY_SCORE) from CHICAGO_PUBLIC_SCHOOLS #99
%sql select NAME_OF_SCHOOL from CHICAGO_PUBLIC_SCHOOLS where SAFETY_SCORE =99

# 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance
%sql select NAME_OF_SCHOOL, average_student_attendance from CHICAGO_PUBLIC_SCHOOLS \
    ORDER BY average_student_attendance nulls last limit 5

# Get the total College Enrollment for each Community Area
%sql select community_area_name, SUM(college_enrollment) from CHICAGO_PUBLIC_SCHOOLS \
    GROUP by community_area_name

# Get the 5 Community Areas with the least total College Enrollment sorted in ascending order
%sql select community_area_name, SUM(college_enrollment) as SUM_ENROLL from CHICAGO_PUBLIC_SCHOOLS \
    GROUP by community_area_name ORDER by SUM_ENROLL limit 5

# Find the total number of crimes recorded in the CRIME table
%sql select count(*) from CHICAGO_CRIME_DATA

# Retrieve first 10 rows from the CRIME table
%sql select * from CHICAGO_CRIME_DATA fetch first 10 rows only

# How many crimes involve an arrest
%sql select count(*) from CHICAGO_CRIME_DATA where arrest = 'TRUE'

# unique types of crimes have been recorded at GAS STATION locations
%sql select distinct(primary_type) from CHICAGO_CRIME_DATA where location_description = 'GAS STATION'

# list all Community Areas whose names start with the letter ‘B
%sql select community_area_name from chicago_sociec where community_area_name LIKE 'B%'

# Which schools in Community Areas 10 to 15 are healthy school certified?
%sql select name_of_school from CHICAGO_PUBLIC_SCHOOLS where (community_area_number BETWEEN 10 and 15) and healthy_school_certified = 'Yes'

# What is the average school Safety Score
%sqp SELECT AVG(safety_score) from CHICAGO_PUBLIC_SCHOOLS

# List the top 5 Community Areas by average College Enrollment number of students
%sql SELECT community_area_name, AVG(college_enrollment) as avg_enroll from CHICAGO_PUBLIC_SCHOOLS \
    GROUP by community_area_name ORDER by ave_enroll DESC nulls last limit 5
                                                              
# Use a sub-query to determine which Community Area has the least value for school Safety Score
%sql SELECT community_area_name, safety_score from CHICAGO_PUBLIC_SCHOOLS \
    where safey_score = (SELECT MIN(safety_score) from CHICAGO_PUBLIC_SCHOOLS)

                                                              
# Without using an explicit JOIN operator, Find the Per Capita Income of the Community Area which has a school Safety Score of 1.
%sql SELECT community_area_name, per_capita_income, from chicago_soceco as csd \
    where community_area_name in (SELECT community_area_name from CHICAGO_PUBLIC_SCHOOLS where safey_score = 1)
%sql SELECT csd.community_area_name, csd.per_capita_income, from chicago_soceco as csd, CHICAGO_PUBLIC_SCHOOLS as cps \
    where csd.community_area_name = cps.community_area_name and cps.safey_score = 1



UsageError: Line magic function `%sql` not found.


### 3) Additional functions

In [ ]:
# nulls last: exclude NONE/nulls values

# top 10 schools with the highest "Average Student Attendance" 
%sql select NAME_OF_SCHOOL, average_student_attendance from CHICAGO_PUBLIC_SCHOOLS \
    ORDER BY average_student_attendance DESC nulls last limit 10  
# limit 10 = fetch first 10 rows only
                                                    

In [ ]:
# Replace function

# remove the '%' sign from the above result set for Average Student Attendance column
%sql select NAME_OF_SCHOOL, REPLACE(average_student_attendance,'%','') from CHICAGO_PUBLIC_SCHOOLS \
    ORDER BY average_student_attendance nulls last limit 5
                                                                     

In [ ]:
# Cast/Desimal function (convert char column type to numeric): CAST("Column_Name" as DOUBLE), DECIMAL("Column_Name")

# which Schools have Average Student Attendance lower than 70%?
%sql select NAME_OF_SCHOOL, average_student_attendance from CHICAGO_PUBLIC_SCHOOLS \
    where CAST(REPLACE(average_student_attendance,'%','') AS DOUBLE) < 70


### 4) Two tables

In [ ]:
# Get the hardship index for the community area which has College Enrollment of 4638
%sql select cps.community_area_name, csd.hardship_index from CHICAGO_SOCIOECONOMIC_DATA as csd, \
    CHICAGO_PUBLIC_SCHOOLS as cps \
    where cps.community_area_number = csd.ca and college_enrollment = 4638

# Get the hardship index for the community area which has the highest value for College Enrollment
%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in ( select community_area_number from CHICAGO_PUBLIC_SCHOOLS order by college_enrollment desc limit 1 )
# take advantage of "order by + limit 1"